In [3]:
from fbprophet import Prophet

import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt

from ipywidgets import widgets
from IPython.display import display

In [8]:
n=500

growth_rate = widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.01, description='growth_rate')
offset=widgets.IntSlider(min=-100, max=100, step=10, value=50, description='offset')

capacity1=widgets.IntSlider(min=-150, max=150, step=10, value=100, description='capacity1')
capacity_change_index=widgets.IntSlider(min=1, max=n-1, step=1, value=250, description='capacity change index')
capacity2=widgets.IntSlider(min=-150, max=150, step=10, value=120, description='capacity2')

rate_change1 = widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.02, description='rate_change1')
rate_change2 = widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=-0.02, description='rate_change2')

ui = widgets.VBox([
                   widgets.HBox([offset, growth_rate]),
                   widgets.HBox([capacity1, capacity_change_index, capacity2]),
                   widgets.HBox([rate_change1, rate_change2])
])

def f(offset, growth_rate, capacity1=100, capacity_change_index=250, capacity2=120, rate_change1=0.02, rate_change2=-0.02):
    t = pd.DataFrame({'t':range(n)})
    t['t-offset'] = t['t'] - offset
    t['growth_rate*(t-offset)'] = growth_rate*(t['t'] - offset)
    t['1+exp(-growth_rate*(t-offset))'] = 1 + t['growth_rate*(t-offset)'].apply(lambda x: math.exp(-x))
    t['1/(1+exp(-growth_rate*(t-offset)))'] = 1/t['1+exp(-growth_rate*(t-offset))']
    t['capacity'] = t['t'].apply(lambda x: capacity1 if x < capacity_change_index else capacity2)
    t['logistic_growth'] = t['capacity']/t['1+exp(-growth_rate*(t-offset))']
    
    changepoints = np.random.choice(t['t'], 2)
    changepoints.sort()

    rate_changes = np.reshape(np.repeat(np.array([rate_change1, rate_change2]), t['t'].shape[0]), (2, t['t'].shape[0]))
    a = (np.array(t['t']) >= np.reshape(np.repeat(changepoints, t['t'].shape[0]), (2, t['t'].shape[0]))).astype(int)
    adjustment_1 = (changepoints[0] - offset) * (1 - (growth_rate)/(growth_rate + rate_change1))
    adjustment_2 = (changepoints[1] - offset - adjustment_1) * (1 - (growth_rate + rate_change1)/(growth_rate + rate_change1 + rate_change2))
    adjustments = np.reshape(np.repeat(np.array([adjustment_1, adjustment_2]), t['t'].shape[0]), (2, t['t'].shape[0]))

    t['total_growth_rate'] = growth_rate + np.sum(rate_changes*a, axis=0)
    t['adjustments'] = (offset + np.sum(adjustments*a, axis=0))
    t['1+exp(-total_growth_rate*(t-total_offset))'] = (t['total_growth_rate'] * (t['t'] - t['adjustments'])).apply(lambda x: 1+math.exp(-x))
    t['logistic_growth_with_changepoints'] = t['capacity'] / t['1+exp(-total_growth_rate*(t-total_offset))']

    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,10))

    t[['growth_rate*(t-offset)', '1+exp(-growth_rate*(t-offset))', '1/(1+exp(-growth_rate*(t-offset)))']].plot(ax=ax1, title='Logistic growth components')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)

    t[['logistic_growth']].plot(ax=ax2, title='Logistic growth')
    ax2.get_legend().remove()
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)

    t[['total_growth_rate', '1+exp(-total_growth_rate*(t-total_offset))']].plot(ax=ax3, title='Logistic growth with 2 changepoints components')
    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)

    t[['logistic_growth_with_changepoints']].plot(ax=ax4, title='Logistic growth with 2 changepoints')
    ax4.get_legend().remove()
    ax4.spines['top'].set_visible(False)
    ax4.spines['right'].set_visible(False)

out = widgets.interactive_output(f, {'offset': offset, 'growth_rate': growth_rate, 
                                     'capacity1': capacity1, 'capacity_change_index': capacity_change_index, 'capacity2': capacity2,
                                     'rate_change1': rate_change1, 'rate_change2': rate_change2})

display(ui, out)

Output()